In [1]:
import nltk


import logging

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import cm
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import (RepeatedStratifiedKFold, cross_val_score, )
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
import re

from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score


import spacy

import pandas as pd

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim import models

# spacy for lemmatization
import spacy

# Plotting tools
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

#############################################

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
workers = 4 #Numbers of workers throughout the project

use_umap = False #make this True if you want to use UMAP for your visualizations

min_df = 5 #Minimum number for document frequency in the corpus
rm_top = 5 #Remove top n frequent words

from nltk import bigrams
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
nlp = spacy.load('en_core_web_sm')

In [6]:
dataset_test = fetch_20newsgroups(shuffle=True, remove=('headers', 'footers', 'qutes'), subset='train')

In [7]:
# put your data into a dataframe
news_df = pd.DataFrame({'News': dataset_test.data,
                       'Target': dataset_test.target})

# get dimensions of data 
news_df.shape

(11314, 2)

In [8]:
news_df['Target_name'] = news_df['Target'].apply(lambda x: dataset_test.target_names[x])

In [9]:
news_df.head()

,News,Target,Target_name
0,I was wondering if anyone out there could enli...,7,rec.autos
1,A fair number of brave souls who upgraded thei...,4,comp.sys.mac.hardware
2,"well folks, my mac plus finally gave up the gh...",4,comp.sys.mac.hardware
3,Robert J.C. Kyanko (rob@rjck.UUCP) wrote:\n> a...,1,comp.graphics
4,"From article <C5owCB.n3p@world.std.com>, by to...",14,sci.space


In [10]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Emails
docs = [re.sub('\S*@\S*\s?', '', sent) for sent in news_df.News]

# Remove new line characters
docs = [re.sub('\s+', ' ', sent) for sent in docs]

# Remove distracting single quotes
docs = [re.sub("\'", "", sent) for sent in docs]


data_words = list(sent_to_words(docs))

bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)
# Create Corpus
texts = data_lemmatized
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

TFIDF = models.TfidfModel(corpus) # Fit TF-IDF model
trans_TFIDF = TFIDF[corpus]

In [13]:
print(data_lemmatized[:1])

[['wonder', 'enlighten', 'car', 'saw', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'bricklin', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail']]


In [14]:
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 4), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 2), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)], [(5, 2), (31, 1), (32, 2), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 2), (57, 1), (58, 1), (59, 1), (60, 1), (61, 1), (62, 1), (63, 2), (64, 1), (65, 1), (66, 1), (67, 2), (68, 1)]]


In [15]:
SEED = 100 # Set random seed
NUM_topics = 10 # Set number of topics
ALPHA = 50/NUM_topics # Alpha being set as 50/k gives consistant smooth results.
ETA = 0.1 # Set eta

In [16]:
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=trans_TFIDF,
                                       id2word=id2word,
                                       num_topics=NUM_topics, 
                                       random_state=SEED,
                                       chunksize=100,
                                       passes=50,
                                       per_word_topics=True,
                                       alpha=ALPHA,
                                       eta=ETA,)


print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -10.109023772539299

Coherence Score:  0.39068808670328786


In [17]:
print(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0, '0.005*"find" + 0.005*"go" + 0.005*"way" + 0.005*"many" + 0.004*"point" + 0.004*"space" + 0.004*"talk" + 0.004*"reason" + 0.004*"far" + 0.003*"still"'), (1, '0.004*"buy" + 0.004*"new" + 0.004*"drive" + 0.004*"gun" + 0.004*"kill" + 0.004*"price" + 0.003*"car" + 0.003*"machine" + 0.003*"else" + 0.003*"window"'), (2, '0.005*"team" + 0.005*"game" + 0.005*"try" + 0.004*"play" + 0.004*"want" + 0.004*"win" + 0.004*"back" + 0.004*"player" + 0.004*"however" + 0.004*"much"'), (3, '0.005*"mail" + 0.005*"information" + 0.005*"send" + 0.004*"email" + 0.004*"help" + 0.004*"number" + 0.004*"work" + 0.004*"wonder" + 0.004*"address" + 0.003*"also"'), (4, '0.006*"window" + 0.005*"program" + 0.005*"problem" + 0.005*"line" + 0.004*"computer" + 0.004*"color" + 0.004*"order" + 0.004*"monitor" + 0.004*"version" + 0.003*"mac"'), (5, '0.004*"sure" + 0.004*"true" + 0.004*"law" + 0.003*"chip" + 0.003*"thing" + 0.003*"old" + 0.003*"show" + 0.003*"life" + 0.003*"week" + 0.003*"child"'), (6, '0.005*"first" + 0

In [18]:

# train is now 75% of the entire data set
# the _junk suffix means that we drop that variable completely
train, test = train_test_split(news_df, train_size=0.6)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
test, val = train_test_split(test, test_size=0.5) 

print(train.shape, test.shape, val.shape)

(6788, 3) (2263, 3) (2263, 3)


In [19]:
# Remove Emails
docs_test = [re.sub('\S*@\S*\s?', '', sent) for sent in test.News]

# Remove new line characters
docs_test = [re.sub('\s+', ' ', sent) for sent in docs_test]

# Remove distracting single quotes
docs_test = [re.sub("\'", "", sent) for sent in docs_test]


data_words_test = list(sent_to_words(docs_test))

bigram = gensim.models.Phrases(data_words_test, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_test], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops_test = remove_stopwords(data_words_test)

# Form Bigrams
data_words_bigrams_test = make_bigrams(data_words_nostops_test)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_test = lemmatization(data_words_bigrams_test, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word_test = corpora.Dictionary(data_lemmatized_test)
# Create Corpus
texts_test = data_lemmatized_test
# Term Document Frequency
corpus_test = [id2word_test.doc2bow(text) for text in texts_test]

TFIDF_test = models.TfidfModel(corpus_test) # Fit TF-IDF model
trans_TFIDF_test = TFIDF_test[corpus_test]






In [20]:
# Remove Emails
docs_val = [re.sub('\S*@\S*\s?', '', sent) for sent in val.News]

# Remove new line characters
docs_val = [re.sub('\s+', ' ', sent) for sent in docs_val]

# Remove distracting single quotes
docs_val = [re.sub("\'", "", sent) for sent in docs_val]


data_words_val = list(sent_to_words(docs_val))

bigram = gensim.models.Phrases(data_words_val, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_val], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops_val = remove_stopwords(data_words_val)

# Form Bigrams
data_words_bigrams_val = make_bigrams(data_words_nostops_val)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_val = lemmatization(data_words_bigrams_val, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word_val = corpora.Dictionary(data_lemmatized_val)
# Create Corpus
texts_val = data_lemmatized_val
# Term Document Frequency
corpus_val = [id2word_val.doc2bow(text) for text in texts_val]

TFIDF_val = models.TfidfModel(corpus_val) # Fit TF-IDF model
trans_TFIDF_val = TFIDF_val[corpus_val]



In [21]:
# Remove Emails
docs_train = [re.sub('\S*@\S*\s?', '', sent) for sent in train.News]

# Remove new line characters
docs_train = [re.sub('\s+', ' ', sent) for sent in docs_train]

# Remove distracting single quotes
docs_train = [re.sub("\'", "", sent) for sent in docs_train]


data_words_train = list(sent_to_words(docs_train))

bigram = gensim.models.Phrases(data_words_train, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words_train], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# Remove Stop Words
data_words_nostops_train = remove_stopwords(data_words_train)

# Form Bigrams
data_words_bigrams_train = make_bigrams(data_words_nostops_train)


# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_train = lemmatization(data_words_bigrams_train, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Create Dictionary
id2word_train = corpora.Dictionary(data_lemmatized_train)
# Create Corpus
texts_train = data_lemmatized_train
# Term Document Frequency
corpus_train = [id2word_train.doc2bow(text) for text in texts_train]


TFIDF_train = models.TfidfModel(corpus_train) # Fit TF-IDF model
trans_TFIDF_train = TFIDF_train[corpus_train]

In [44]:
SEED = 100 # Set random seed
NUM_topics = 50 # Set number of topics
ALPHA = 50/NUM_topics # Alpha being set as 50/k gives consistant smooth results.
ETA = 0.1 # Set eta

In [61]:
# Build LDA model
from gensim.models import LdaModel
lda_train =  gensim.models.LdaMulticore(corpus=trans_TFIDF,
                                       id2word=id2word,
                                       num_topics= NUM_topics, 
                                       random_state= SEED,
                                       per_word_topics=True,
                                       chunksize=100,
                                        alpha= ALPHA,
                                        eta= ETA,
                                        passes=20
                                      )


In [52]:
def document_to_lda_features(lda_model, document):
    """ Transforms a bag of words document to features.
    It returns the proportion of how much each topic was
    present in the document.
    """
    topic_importances = lda_train.get_document_topics(document, minimum_probability=0)
    topic_importances = np.array(topic_importances)
    return topic_importances[:,1]


In [62]:
vec = list(map(lambda doc:
                                      document_to_lda_features(lda_train, 
                                                              doc),
                                       trans_TFIDF))

In [53]:
val_vec = list(map(lambda doc:
                                      document_to_lda_features(lda_train, 
                                                              doc),
                                       trans_TFIDF_val))
test_vec = list(map(lambda doc:
                                      document_to_lda_features(lda_train, 
                                                              doc),
                                      trans_TFIDF_test))
train_vec = list(map(lambda doc:
                                      document_to_lda_features(lda_train, 
                                                              doc),
                                      trans_TFIDF_train))

In [63]:
X_vec = np.array(list(map(np.array, vec)))
X_scale = scaler.fit_transform(X_vec)

Xtrain, Xtest = train_test_split(X_scale, train_size=0.6)

# test is now 10% of the initial data set
# validation is now 15% of the initial data set
Xtest, Xval = train_test_split(Xtest, test_size=0.5) 

print(Xtrain.shape, Xtest.shape, Xval.shape)

(6788, 50) (2263, 50) (2263, 50)


In [54]:
# since train_data['lda_features'] don't have the needed shape and type yet,
# we first have to transform every entry
X_train_lda = np.array(list(map(np.array, train_vec)))
X_val_lda = np.array(list(map(np.array, val_vec)))
X_test_lda = np.array(list(map(np.array, test_vec)))

In [55]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train_lda)
X_val_scale = scaler.transform(X_val_lda)
X_test_scale = scaler.transform(X_test_lda)

In [65]:
folds = RepeatedStratifiedKFold(n_splits=5)
clff = MultinomialNB()
lr = LogisticRegression(multi_class='multinomial', solver='newton-cg', penalty='l2')
clf = SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42)
param_grid = {'penalty': ['l1', 'l2']}

params = {}
grid = GridSearchCV(lr, params, cv=folds)

gs_ls = grid.fit(X_train_scale, train.Target)

y_pred = gs_ls.predict(X_val_scale)
print(f1_score(val.Target, y_pred, average='micro'))

print(gs_ls.best_score_)




AttributeError: 'numpy.ndarray' object has no attribute 'Target'